# Semantic Kernel with Azure OpenAI ⚡

This notebook is a simple demonstration of how to use the [Semantic Kernel](https://learn.microsoft.com/semantic-kernel/overview/?WT.mc_id=AZ-MVP-5003203) with [Azure OpenAI](https://azure.microsoft.com/products/ai-services/openai-service?WT.mc_id=AZ-MVP-5003203). 

## Install requirements
Make sure you have configured the necessary settings for the Azure OpenAI Connector. See the [documentation](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md#openai--azure-openai-api-keys).

We only need to install the Semantic Kernel SDK from pypi.org

In [1]:
%pip install semantic-kernel

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached semantic_kernel-1.11.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached pydantic_settings-2.5.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached openai-1.51.0-py3-none-any.whl.metadata (24 kB)
  Using cached openapi_core-0.19.4-py3-none-any.whl.metadata (6.4 kB)
  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached prance-23.6.21.0-py3-none-any.whl.metadata (13 kB)
  Using cached pybars4-0.9.13-py3-none-any.whl
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached typing_extensions-4.12

## Initialize the Kernel

In [4]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = Kernel()
service_id="chat-gpt"

kernel.add_service(
  AzureChatCompletion(
      service_id=service_id,
  )
)

## Add Plugins
The following script will add all plugins from the `plugins_directory` to the kernel:

In [6]:
plugins_directory = "Plugins"
kubernetesPlugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="KubernetesPlugin")

## Run the KubectlPrompt Plugin 

In [7]:
result = await kernel.invoke(kubernetesPlugin["KubectlPrompt"], input="list all namespace")
print(result)

```sh
kubectl get namespaces
```


## Run the KubeMatch Plugin

In [8]:
goodFit="""
This project involves developing a high-traffic e-commerce platform that expects dynamic scaling and high availability to handle seasonal peaks in user traffic. 
The architecture is based on microservices, each responsible for different aspects such as user authentication, product catalog, and payment processing. 
The team is experienced in containerized environments and requires a solution that allows for efficient deployment, scaling, and management of services, with robust orchestration capabilities.
"""

result = await kernel.invoke(kubernetesPlugin["KubeMatch"], input=goodFit)
print(result)

Based on the provided project description and the factors to consider, Kubernetes appears to be a highly suitable platform for deploying the high-traffic e-commerce platform. Here's a detailed assessment:

### Scalability
**Requirement:** The project needs to scale dynamically to handle seasonal peaks in user traffic.
**Kubernetes Fit:** Kubernetes excels in dynamic scaling. It supports horizontal pod autoscaling, which can automatically adjust the number of running pods based on CPU utilization or other select metrics. This makes it ideal for handling fluctuating traffic patterns.

### Manageability
**Requirement:** Centralized platform to manage containers.
**Kubernetes Fit:** Kubernetes provides a centralized platform for managing containerized applications. It offers features like namespaces, labels, and annotations to organize and manage resources efficiently. Tools like the Kubernetes Dashboard and third-party solutions (e.g., Rancher) further enhance manageability.

### Microser

In [9]:

badFit="""
This project is a small-scale internal tool for automating office inventory management, expected to have a stable and predictable load with minimal scalability requirements. 
The application is built with a simple, monolithic architecture and will be deployed on a few servers. 
The team lacks experience with containerization and Kubernetes, and the simplicity of the project does not justify the overhead and complexity of implementing a Kubernetes environment.
"""

result = await kernel.invoke(kubernetesPlugin["KubeMatch"], input=badFit)
print(result)

Based on the provided project description and the factors to consider, here is an assessment of whether Kubernetes is a suitable platform for deployment:

### Scalability
- **Assessment**: The project has minimal scalability requirements and is expected to have a stable and predictable load.
- **Conclusion**: Kubernetes is designed for dynamic scaling, which is not necessary for this project. Therefore, Kubernetes may be overkill for the scalability needs of this project.

### Manageability
- **Assessment**: While Kubernetes offers centralized management for containers, the project is small-scale and involves a simple monolithic architecture.
- **Conclusion**: The benefits of Kubernetes' centralized management are less significant for a small-scale project with minimal complexity.

### Microservices Architecture
- **Assessment**: The project is built with a monolithic architecture.
- **Conclusion**: Kubernetes excels in managing microservices architectures, but this project does not fo

## Add native code as a plugin
The plugin described below allows us to **execute kubectl** commands.

In [10]:
import subprocess

from semantic_kernel.functions import kernel_function

class ExecuteKubectlPlugin:
    """
    Description: This plugin is used to execute kubectl commands
    """

    @kernel_function(
        description="Execute kubectl command",
        name="ExecuteKubectl",
    )
    def execute_kubectl_command (self, input: str) -> str:
        """
        Execute kubectl command
        """
        try:
            command = input.replace("kubectl", "").strip() # remove kubectl from the input
            result = subprocess.run(['kubectl'] + command.split(), capture_output=True, text=True, check=True)

            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"Error executing kubectl command: {e}")


### Add the native plugin to the kernel

In [11]:
execute_kubectl_plugin = kernel.add_plugin(plugin=ExecuteKubectlPlugin(), plugin_name="ExecuteKubectlPlugin")

In [12]:
execute_kubectl_fnc = execute_kubectl_plugin["ExecuteKubectl"]
kubectl_output = await execute_kubectl_fnc(kernel,input="kubectl cluster-info")
print(kubectl_output)


Kubernetes control plane is running at https://kufstein-demo-dns-iedm2rw6.hcp.germanywestcentral.azmk8s.io:443
CoreDNS is running at https://kufstein-demo-dns-iedm2rw6.hcp.germanywestcentral.azmk8s.io:443/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy
Metrics-server is running at https://kufstein-demo-dns-iedm2rw6.hcp.germanywestcentral.azmk8s.io:443/api/v1/namespaces/kube-system/services/https:metrics-server:/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.




## Using the Planner 🪄

In [ ]:
from semantic_kernel.planners.function_calling_stepwise_planner import (
    FunctionCallingStepwisePlanner,
    FunctionCallingStepwisePlannerOptions,
)

question = "list all namepsaces"

options = FunctionCallingStepwisePlannerOptions(
    max_iterations=10,
    max_tokens=4000,
)

planner = FunctionCallingStepwisePlanner(service_id=service_id, options=options)
result = await planner.invoke(kernel, question)

### Print the plan

In [ ]:
print(f"Chat history: {result.chat_history}\n")